In [ ]:
# default_exp core

# pyshell

> A Shell object that allows the user to access every program in the `PATH` as an object method. 

In [ ]:
#hide
from nbdev.showdoc import *

## Imports

In [ ]:
#export
import subprocess
from fastcore.basics import *
from fastcore.all import *

## Command

In [ ]:
#export
class Command:
    """A shell command."""
    def __init__(self, name): self.name = name
    def __call__(self, *args, shell=True, decode='utf-8', out_error=False, **kwargs):
        command = [self.name]
        if 'flags' in kwargs: 
            command.append('-'+kwargs.pop('flags'))
        for k in kwargs:
            command.append('--' + k.replace('_','-'))
            command.append(str(kwargs[k]))
        command.extend([str(x) for x in args])
        process = subprocess.Popen(' '.join(command), stdout=subprocess.PIPE, shell=shell, stderr=subprocess.PIPE)
        output, error = process.communicate()
        if error:
            error = error.decode("utf-8").strip('\n')
            if out_error:
                return error
            else:
                raise Exception(error)
        if decode:
            output = output.decode("utf-8").strip('\n')
        return output

In [ ]:
ls = Command('ls')
test_eq(ls(), '00_core.ipynb\nautocomplete.png\nCONTRIBUTING.md\ndocker-compose.yml\ndocs\nindex.ipynb\nLICENSE\nMakefile\nMANIFEST.in\nmanpage.png\npyshell\nREADME.md\nsettings.ini\nsetup.py')

## Shell

In [ ]:
#export
class Shell:
    def __getattr__(self, k, *args, **kwargs):
        if k[0]=='_': return super().__getattr__(k)
        com = Command(k)
        com.__doc__ = Command('man')(k, out_error=True)
        return com 
    def _dir(self): return L(L(self.echo('$PATH').split(':')).map(self.ls).map(Self.split('\n')).sum()).unique().filter()
    def __dir__(self): return custom_dir(self, self._dir())

In [ ]:
sh = Shell()

In [ ]:
test_eq(sh.ls(), '00_core.ipynb\nautocomplete.png\nCONTRIBUTING.md\ndocker-compose.yml\ndocs\nindex.ipynb\nLICENSE\nMakefile\nMANIFEST.in\nmanpage.png\npyshell\nREADME.md\nsettings.ini\nsetup.py')

In [ ]:
test_eq(sh.ls('\.giti*', flags='a'), '.gitignore')

## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
